In [1]:
# import necessary library
import pandas as pd
import math
import seaborn as sns
from matplotlib import pyplot as plt
import re
import numpy as np
import geopandas as gpd
from shapely.geometry import Point # Point class
from shapely.geometry import shape
import networkx as nx
import osmnx as ox
import os 
from geopandas import GeoDataFrame, read_file
from geopandas.tools import overlay
import shapely.geometry as geom
import shapely.wkt
from shapely.geometry import  Point, mapping
from itertools import combinations
import fiona

C:\Users\Md Mintu Miah\anaconda3\envs\geo_env\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Task 1: Read the Poisson Regression Model Annual Total traffic Data


### Task 2: Identify the intersection and aggregate the intersection total Annual Total traffic


### Task 3: Export the Intersection Volume file as GeoJSON file and share with Iman.

In [2]:
# read the Poisson Regression Model prediction AADBT file
predicted_traffic=gpd.read_file('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Modeling_Sep22,2023/SHS_AADBT_Prediction_Uisng_PR_Model_Dec21_2023.json')
predicted_traffic.head()

,AADBT,index,tdg_id,county,street_name,shs_status,Length,class,Stv_adb,empnum_density_t,exist_bike_parking_q,Intersection_density5,pctnoveh_h,pctbiketowork_h,urban,Yolo,log_Stv_adb,geometry
0,36.923849,0,6496.0,Del Norte,SR-101-DEL NORTE,SHS,0.016049,low,0.000000,0.000000,0.0,0.0,0.027174,0.0,0,0,-4.605170,"LINESTRING (-124.20812 41.99626, -124.20811 41..."
1,85.956132,1,6497.0,Del Norte,SR-101-DEL NORTE,SHS,0.036890,low,2.123288,0.000000,0.0,0.0,0.027174,0.0,0,0,0.757664,"LINESTRING (-124.20811 41.99602, -124.20808 41..."
2,36.923849,2,6498.0,Del Norte,SR-101-DEL NORTE,SHS,0.016515,low,0.000000,0.000000,0.0,0.0,0.027174,0.0,0,0,-4.605170,"LINESTRING (-124.20780 41.99624, -124.20812 41..."
3,86.129133,3,6500.0,Del Norte,SR-101-DEL NORTE,SHS,0.199039,low,2.150685,159.154943,0.0,0.0,0.027174,0.0,0,0,0.770425,"LINESTRING (-124.20789 41.99231, -124.20783 41..."
4,45.453235,4,6507.0,Del Norte,NO NAME,XSTREET_SHS,0.189292,low,0.027397,159.154943,0.0,0.0,0.027174,0.0,0,0,-3.286158,"LINESTRING (-124.20789 41.99231, -124.20809 41..."


In [3]:
# Convert AADBT to Annual Total traffic. The above prediction is for 2019, as there was 365 days in 2029, we will multiple AADBT with 365 to obtain ATT
predicted_traffic['ATT']=predicted_traffic['AADBT']*365
predicted_traffic.head()

,AADBT,index,tdg_id,county,street_name,shs_status,Length,class,Stv_adb,empnum_density_t,exist_bike_parking_q,Intersection_density5,pctnoveh_h,pctbiketowork_h,urban,Yolo,log_Stv_adb,geometry,ATT
0,36.923849,0,6496.0,Del Norte,SR-101-DEL NORTE,SHS,0.016049,low,0.000000,0.000000,0.0,0.0,0.027174,0.0,0,0,-4.605170,"LINESTRING (-124.20812 41.99626, -124.20811 41...",13477.204780
1,85.956132,1,6497.0,Del Norte,SR-101-DEL NORTE,SHS,0.036890,low,2.123288,0.000000,0.0,0.0,0.027174,0.0,0,0,0.757664,"LINESTRING (-124.20811 41.99602, -124.20808 41...",31373.988080
2,36.923849,2,6498.0,Del Norte,SR-101-DEL NORTE,SHS,0.016515,low,0.000000,0.000000,0.0,0.0,0.027174,0.0,0,0,-4.605170,"LINESTRING (-124.20780 41.99624, -124.20812 41...",13477.204780
3,86.129133,3,6500.0,Del Norte,SR-101-DEL NORTE,SHS,0.199039,low,2.150685,159.154943,0.0,0.0,0.027174,0.0,0,0,0.770425,"LINESTRING (-124.20789 41.99231, -124.20783 41...",31437.133506
4,45.453235,4,6507.0,Del Norte,NO NAME,XSTREET_SHS,0.189292,low,0.027397,159.154943,0.0,0.0,0.027174,0.0,0,0,-3.286158,"LINESTRING (-124.20789 41.99231, -124.20809 41...",16590.430770


In [4]:
predicted_traffic.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
# Lets do the projection
predicted_traffic = predicted_traffic.set_crs("EPSG:4326")
predicted_traffic_proj = predicted_traffic.to_crs(epsg=32610)
predicted_traffic_proj.head()

,AADBT,index,tdg_id,county,street_name,shs_status,Length,class,Stv_adb,empnum_density_t,exist_bike_parking_q,Intersection_density5,pctnoveh_h,pctbiketowork_h,urban,Yolo,log_Stv_adb,geometry,ATT
0,36.923849,0,6496.0,Del Norte,SR-101-DEL NORTE,SHS,0.016049,low,0.000000,0.000000,0.0,0.0,0.027174,0.0,0,0,-4.605170,"LINESTRING (399939.366 4650066.554, 399940.086...",13477.204780
1,85.956132,1,6497.0,Del Norte,SR-101-DEL NORTE,SHS,0.036890,low,2.123288,0.000000,0.0,0.0,0.027174,0.0,0,0,0.757664,"LINESTRING (399940.086 4650040.735, 399941.950...",31373.988080
2,36.923849,2,6498.0,Del Norte,SR-101-DEL NORTE,SHS,0.016515,low,0.000000,0.000000,0.0,0.0,0.027174,0.0,0,0,-4.605170,"LINESTRING (399965.852 4650064.337, 399939.366...",13477.204780
3,86.129133,3,6500.0,Del Norte,SR-101-DEL NORTE,SHS,0.199039,low,2.150685,159.154943,0.0,0.0,0.027174,0.0,0,0,0.770425,"LINESTRING (399952.770 4649627.937, 399956.489...",31437.133506
4,45.453235,4,6507.0,Del Norte,NO NAME,XSTREET_SHS,0.189292,low,0.027397,159.154943,0.0,0.0,0.027174,0.0,0,0,-3.286158,"LINESTRING (399952.770 4649627.937, 399936.207...",16590.430770


In [7]:
# Identify the intersection point first
#extract shapely geometries
multiline = predicted_traffic_proj['geometry']

#create empty list of points
points = []

#find intersections and append to points list
for line1, line2 in combinations([line for line in multiline],2):
    if line1.intersects(line2):
        points.append(line1.intersection(line2))
len(points)

77177

In [111]:
# Form the data frame with the identified points
intersection_pts = gpd.gpd.GeoDataFrame({'geometry':points})
intersection_pts.drop_duplicates('geometry', inplace=True)
intersection_pts['inter_id']=intersection_pts.index
intersection_pts.head()

,geometry,inter_id
0,POINT (399940.086 4650040.735),0
1,POINT (399939.366 4650066.554),1
5,POINT (399941.950 4649981.395),5
8,POINT (399952.770 4649627.937),8
10,POINT (399997.204 4649312.085),10


In [112]:
crs = {'init': 'epsg:32610'}
intersection_pts_s=gpd.GeoDataFrame(intersection_pts, crs=crs, geometry = 'geometry')
intersection_pts_s['inter_id']=intersection_pts_s.index
intersection_pts_s.head()

C:\Users\Md Mintu Miah\anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,geometry,inter_id
0,POINT (399940.086 4650040.735),0
1,POINT (399939.366 4650066.554),1
5,POINT (399941.950 4649981.395),5
8,POINT (399952.770 4649627.937),8
10,POINT (399997.204 4649312.085),10


In [94]:
intersection_pts.shape

(24174, 2)

In [22]:
# It seems that we have 24,174 intersections in state highway system
# lets import this file as geojson and check the in GIS visulization, make sure epsg is same with polyline shape file for visualization
intersection_pts_c = intersection_pts.set_crs("EPSG:32610")
intersection_pts_save=intersection_pts_c.to_crs(epsg=4326)
crs = {'init': 'epsg:4326'}
shs_intersection= gpd.GeoDataFrame(intersection_pts_save, crs=crs, geometry = 'geometry')
shs_intersection.to_file('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Modeling_Sep22,2023/SHS_intersection.json', driver="GeoJSON")

C:\Users\Md Mintu Miah\anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [96]:
intersection_pts_c.crs

<Derived Projected CRS: EPSG:32610>
Name: WGS 84 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 126°W and 120°W, northern hemisphere between equator and 84°N, onshore and offshore. Canada - British Columbia (BC); Northwest Territories (NWT); Nunavut; Yukon. United States (USA) - Alaska (AK).
- bounds: (-126.0, 0.0, -120.0, 84.0)
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [97]:
shs_intersection.head()

,geometry,inter_id
0,POINT (-124.208 41.996),0
1,POINT (-124.208 41.996),1
5,POINT (-124.208 41.995),5
8,POINT (-124.208 41.992),8
10,POINT (-124.207 41.989),10


In [98]:
# Now create a 10 meter buffer to identify all the links corresponding to intersection and sum ATT for each buffer
intersection_pts['geometry']=intersection_pts['geometry'].buffer(10)
intersection_pts.head()

,geometry,inter_id
0,"POLYGON ((399950.086 4650040.735, 399950.038 4...",0
1,"POLYGON ((399949.366 4650066.554, 399949.317 4...",1
5,"POLYGON ((399951.950 4649981.395, 399951.902 4...",5
8,"POLYGON ((399962.770 4649627.937, 399962.722 4...",8
10,"POLYGON ((400007.204 4649312.085, 400007.156 4...",10


In [99]:
# Assign index as the intersection id
intersection_pts['buffer_id']=intersection_pts.index
intersection_pts.head()

,geometry,inter_id,buffer_id
0,"POLYGON ((399950.086 4650040.735, 399950.038 4...",0,0
1,"POLYGON ((399949.366 4650066.554, 399949.317 4...",1,1
5,"POLYGON ((399951.950 4649981.395, 399951.902 4...",5,5
8,"POLYGON ((399962.770 4649627.937, 399962.722 4...",8,8
10,"POLYGON ((400007.204 4649312.085, 400007.156 4...",10,10


In [41]:
intersection_pts.shape

(24174, 3)

In [100]:
# assign crs
crs = {'init': 'epsg:32610'}
intersection_pts1= gpd.GeoDataFrame(intersection_pts, crs=crs, geometry = 'geometry')

C:\Users\Md Mintu Miah\anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [101]:
# Now  Join with road shape file and sum the ATT
buffer_link=gpd.sjoin(predicted_traffic_proj,intersection_pts1, how="inner", op='intersects')
buffer_link.head()

C:\Users\Md Mintu Miah\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3488: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,AADBT,index,tdg_id,county,street_name,shs_status,Length,class,Stv_adb,empnum_density_t,...,pctnoveh_h,pctbiketowork_h,urban,Yolo,log_Stv_adb,geometry,ATT,index_right,inter_id,buffer_id
0,36.923849,0,6496.0,Del Norte,SR-101-DEL NORTE,SHS,0.016049,low,0.000000,0.0,...,0.027174,0.0,0,0,-4.605170,"LINESTRING (399939.366 4650066.554, 399940.086...",13477.20478,0,0,0
1,85.956132,1,6497.0,Del Norte,SR-101-DEL NORTE,SHS,0.036890,low,2.123288,0.0,...,0.027174,0.0,0,0,0.757664,"LINESTRING (399940.086 4650040.735, 399941.950...",31373.98808,0,0,0
5,36.923849,5,6515.0,Del Norte,SR-101-DEL NORTE,SHS,0.018877,low,0.000000,0.0,...,0.027174,0.0,0,0,-4.605170,"LINESTRING (399940.086 4650040.735, 399924.951...",13477.20478,0,0,0
0,36.923849,0,6496.0,Del Norte,SR-101-DEL NORTE,SHS,0.016049,low,0.000000,0.0,...,0.027174,0.0,0,0,-4.605170,"LINESTRING (399939.366 4650066.554, 399940.086...",13477.20478,1,1,1
2,36.923849,2,6498.0,Del Norte,SR-101-DEL NORTE,SHS,0.016515,low,0.000000,0.0,...,0.027174,0.0,0,0,-4.605170,"LINESTRING (399965.852 4650064.337, 399939.366...",13477.20478,1,1,1


In [102]:
# Group by 'buffer id' and aggregate 'road_name' as a list
inter_name = buffer_link.groupby('buffer_id')['street_name'].apply(list).reset_index()
# Create a new column 'intersection_name' containing unique road names
inter_name['intersection_name'] = inter_name['street_name'].apply(lambda x: list(set(x)))
inter_name.tail(10)

,buffer_id,street_name,intersection_name
24164,77009,"[West 4th Street, South Butte Street]","[South Butte Street, West 4th Street]"
24165,77039,"[El Camino Avenue, El Camino Avenue]",[El Camino Avenue]
24166,77058,"[Cabrillo Highway, Cabrillo Highway, Cabrillo ...","[Cabrillo Highway, Coast Road]"
24167,77081,"[Weedpatch Highway, Weedpatch Highway]",[Weedpatch Highway]
24168,77105,"[Willow Street, McKittrick Highway, McKittrick...","[McKittrick Highway, Willow Street]"
24169,77125,"[Aerospace Highway, Midland Trail]","[Midland Trail, Aerospace Highway]"
24170,77126,"[East Ridgecrest Boulevard, West Ridgecrest Bo...","[West Ridgecrest Boulevard, North China Lake B..."
24171,77134,"[SR-395-MONO, El Camino Sierra]","[El Camino Sierra, SR-395-MONO]"
24172,77140,"[Lucille Avenue, Lincoln Boulevard, Lincoln Bo...","[Lucille Avenue, Lincoln Boulevard]"
24173,77159,"[Coolidge Street, East Riverside Drive, East R...","[East Riverside Drive, Coolidge Street]"


In [46]:
inter_name.shape

(24174, 3)

In [103]:
# Now calculate the total intersection volume
inter_vol=buffer_link.groupby(['buffer_id'])['ATT'].sum().reset_index()
inter_vol.head()

,buffer_id,ATT
0,0,58328.397639
1,1,40431.614340
2,5,94121.964239
3,8,74711.654852
4,10,104010.660675


In [48]:
inter_vol.shape, buffer_link.shape

((24174, 2), (74981, 22))

In [113]:
# Now we will add the geometry
#inter_vol1=pd.merge(inter_vol,inter_name, left_on=['buffer_id'], right_on=['buffer_id'], how='inner')
inter_vol2=pd.merge(intersection_pts_s,inter_vol, left_on=['inter_id'], right_on=['buffer_id'], how='inner')
inter_vol2.head()

,geometry,inter_id,buffer_id,ATT
0,POINT (399940.086 4650040.735),0,0,58328.397639
1,POINT (399939.366 4650066.554),1,1,40431.614340
2,POINT (399941.950 4649981.395),5,5,94121.964239
3,POINT (399952.770 4649627.937),8,8,74711.654852
4,POINT (399997.204 4649312.085),10,10,104010.660675


In [114]:
inter_vol2.shape

(24174, 4)

In [115]:
inter_vol2.crs

<Derived Projected CRS: EPSG:32610>
Name: WGS 84 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 126°W and 120°W, northern hemisphere between equator and 84°N, onshore and offshore. Canada - British Columbia (BC); Northwest Territories (NWT); Nunavut; Yukon. United States (USA) - Alaska (AK).
- bounds: (-126.0, 0.0, -120.0, 84.0)
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [117]:
# Now save the intersection volume file
#inter_vol3 = inter_vol2.set_crs("EPSG:32610")
inter_vol2_save=inter_vol2.to_crs(epsg=4326)
crs = {'init': 'epsg:4326'}
shs_intersection_vol= gpd.GeoDataFrame(inter_vol2_save, crs=crs, geometry = 'geometry')
shs_intersection_vol.to_file('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Modeling_Sep22,2023/SHS_intersection_total_bike_exposure.json', driver="GeoJSON")

C:\Users\Md Mintu Miah\anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [108]:
# We will also save streetname information as csv
inter_name.to_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Modeling_Sep22,2023/SHS_intersection_name.csv')

# End of the script